<a href="https://colab.research.google.com/github/renatabmagro/RNA_DuracaoCicloMacieira/blob/main/RNA_DuracaoCiclo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Script para implementação de uma rede neural artificial do tipo Multilayer Perceptron para a previsão da duração do ciclo produtivo da macieira a partir de dados meteorológicos.

> Para construir esse código foram utilizados dados meteorológicos + as datas de início da floração e de ínicio da colheita da macieira; ou seja, considerou-se o **ciclo produtivo** como sendo correspondente ao período entre o início da floração e o início da colheita. 

> A planilha contendo os dados necessários para rodar este script pode ser encontrada em: C:\Users\Luciano-PC2\Desktop\Renata_2022\03. Rstudio\Dados\Dados_extra

>> Para comparar o desempenho de diferentes redes neurais, realizou-se a experimentação de:

  * RNA com 1 camada interna;
  * RNA com 3 camadas internas. 

>> Um passo anterior a implementação da RNA foi a análise de correlação entre as variáveis para definir as entrada da rede. Com base nos resultados da análise de correlação, foram selecionadas como variáveis de entrada (inputs): 

*   Temperatura máxima
*   Temperatura mínima 
* Temperatura média 
* Radiação solar 
* Graus-dia
* Graus-dia acumulado 

OBS: os dados meteorológicos são referentes ao período **2008-2022**.

>> A variável de saída (output) corresponde à duração do ciclo (em dias julianos)






In [ ]:
# Instalação das bibliotecas
install.packages("readxl")
library(readxl)
install.packages("tidyr")
install.packages("tidyverse")
library(tidyr)
library(tidyverse)
install.packages("dplyr")
library(dplyr)
install.packages("neuralnet")
library(neuralnet)

In [ ]:
# Leitura dos dados 
dados_fen = read_excel("BancoDadosFen3.xlsx")

# Visualização dados
head(dados_fen)
tail(dados_fen)

In [ ]:
# Estaísticas e características gerais dos dados----
glimpse(dados_fen)

summary(dados_fen)

In [ ]:
# Organização dos dados-----

#Retirada da coluna DiasJulianos - vou usar só a  Duração do ciclo (Dciclo)
#Retirada da variável 'Data' pois não será necessária para a previsão 
#Retirada da variável 'Precip' -> de acordo com os resultados da análise de correlação essa variável apresenta correlação negativa e pode não contribuir para a previsão 
dados_input <- select(dados_fen, -DiasJul, -data, -Precip )

# Visualizar os dados de entrada selecionados 
head(dados_input)

In [7]:
# Divisão do conjunto de dados em treino e teste | treino (70%); teste (30%)-----

set.seed(2022)
split <- sample(1:nrow(dados_input), as.integer(0.7*nrow(dados_input)), F)
#split: sample(x, numero de valores a selecionar, )

train <- dados_input[split,] #dados de treinamento
test <- dados_input[-split,] #dados de test


# Visualizar os dados, caso necessário 
# head(train)
# head(test)

In [9]:
# Normalização dos dados -> método: mínimo e máximo 
  #Esse é uma etapa necessária para evitar que a RNA favoreça determinados dados 

maxs_train <- apply(train, 2, max) 
mins_train <- apply(train, 2, min)

maxs_test <- apply(test, 2, max) 
mins_test <- apply(test, 2, min)


train_set = scale(train, center = mins_train, scale = maxs_train-mins_train)
test_set = scale(test, center = mins_test, scale = maxs_test-mins_test)

# Visualizar os conjuntos de treinamento e teste normalizados, caso necessário 
# head(train_set)
# head(test_set)

In [14]:
# Transformar os dados de teste e treino em um data frame
train_set <- as.data.frame(train_set)
test_set <- as.data.frame(test_set)

In [15]:
# Dividir os dados em variáveis preditoras (x) e previstas (y)
x_train <- select(train_set, -Dciclo)
y_train <- select(train_set, Dciclo)

x_test <- select(test_set, -Dciclo)
y_test <- select(test_set, Dciclo)

In [ ]:
########################################################
##  Implementação da RNA pelo pacote "neuralnet" 
##  Na fórmula da RNA: 
##  o argumento 'linear.output' é utilizado para especificar se queremos realizar uma Regressão => linear.output=TRUE ou uma Classificação => linear.output=FALSE
########################################################

In [11]:
# Definição de uma variável (sigmoid) para a função Sigmoide
sigmoid = function(x) {
  1 / (1 + exp(-x))
}

In [ ]:
# Implementação do modelo com a função sigmoid
  # RNA MLP; 1 camada interna; 3 neurônios internos; função de ativação sigmóide
set.seed(123)
nn_sig <- neuralnet(Dciclo~Tmax+Tmin+Tmed+Rsolar+GD+GDacum,data=cbind(x_train, y_train), 
              hidden=3, act.fct = sigmoid,
              linear.output = TRUE)
plot(nn_sig, rep="best")

In [18]:
# Previsão modelo nn_sig
previsao_nn_sig <- compute(nn_sig, x_test)
pr_nn_sig <- previsao_nn_sig$net.result*(max(dados_input$Dciclo)-min(dados_input$Dciclo))+min(dados_input$Dciclo) #voltando para a escala normal -> resultado da previsão
test.r <- y_test*(max(dados_input$Dciclo)-min(dados_input$Dciclo))+min(dados_input$Dciclo) #voltando p/ a escala normal -> dados de teste

In [ ]:
# Criação dataframe dos resultados do modelo nn_sig (RNA com função sigmoid)
resultados_nn_sig <- data.frame(test.r, pr_nn_sig)
resultados_nn_sig

In [ ]:
# Instalação da biblioteca para calcular as métricas de desempenho do modelo
install.packages("Metrics")
library (Metrics)

In [ ]:
#métricas da previsão com o modelo nn_sig
  # RNA MLP; 1 camada interna; 3 neurônios internos; função de ativação sigmóide
  
#MSE
MSE_nn_sig <- mse(resultados_nn_sig$Dciclo, resultados_nn_sig$pr_nn_sig)

#RMSE
RMSE_nn_sig <- rmse(resultados_nn_sig$Dciclo, resultados_nn_sig$pr_nn_sig)

#MAE
MAE_nn_sig <- mae(resultados_nn_sig$Dciclo, resultados_nn_sig$pr_nn_sig)

print(paste(MSE_nn_sig, RMSE_nn_sig, MAE_nn_sig))

In [ ]:
################################################################################################

In [ ]:
#implementação do modelo com a função sigmoid
  # RNA MLP; 2 camadas interna; 5,2 neurônios internos; função de ativação sigmóide
set.seed(123)
nn_sig2 <- neuralnet(Dciclo~Tmax+Tmin+Tmed+Rsolar+GD+GDacum,data=cbind(x_train, y_train), 
              hidden=c(5,2), rep=5, algorithm="rprop+",
               act.fct = sigmoid,
              linear.output = TRUE)
plot(nn_sig2, rep="best")

In [ ]:
# Previsão modelo nn_sig2
previsao_nn_sig2 <- compute(nn_sig2, x_test)
pr_nn_sig2 <- previsao_nn_sig2$net.result*(max(dados_input$Dciclo)-min(dados_input$Dciclo))+min(dados_input$Dciclo) #voltando para a escala normal -> resultado da previsão
test.r <- y_test*(max(dados_input$Dciclo)-min(dados_input$Dciclo))+min(dados_input$Dciclo) #voltando p/ a escala normal -> dados de teste

# Criação dataframe dos resultados do modelo nn_sig2 
resultados_nn_sig2 <- data.frame(test.r, pr_nn_sig2)
#resultados_nn_sig2

#métricas da previsão com o modelo nn_sig2
  # RNA MLP; 2 camadas interna; 5,2 neurônios internos; função de ativação sigmóide
  
#MSE
MSE_nn_sig2 <- mse(resultados_nn_sig2$Dciclo, resultados_nn_sig2$pr_nn_sig2)

#RMSE
RMSE_nn_sig2 <- rmse(resultados_nn_sig2$Dciclo, resultados_nn_sig2$pr_nn_sig2)

#MAE
MAE_nn_sig2 <- mae(resultados_nn_sig2$Dciclo, resultados_nn_sig2$pr_nn_sig2)

print(paste(MSE_nn_sig2, RMSE_nn_sig2, MAE_nn_sig2))

In [ ]:
#####################################################################################

In [ ]:
# Comparar graficamente modelo nn e nn2 ----- 
par(mfrow=c(1,2))

plot(resultados_nn_sig, col='red',main='Real vs predicted NN',pch=18,cex=0.7)
abline(0,1,lwd=2)
legend('bottomright',legend='NN - 1 hidden layer',pch=18,col='red', bty='n')

plot(resultados_nn_sig2, col='blue',main='Real vs predicted NN2',pch=18,cex=0.7)
abline(0,1,lwd=2)
legend('bottomright',legend='NN2 - 2 hidden layer',pch=18,col='blue', bty='n')

In [ ]:
# Comparar graficamente------
plot(resultados_nn_sig,col='red',main='Real vs predicted NN',pch=18,cex=0.7)
points(resultados_nn_sig2,col='blue',pch=18,cex=0.7)
abline(0,1,lwd=2)
legend('bottomright',legend=c('NN','NN2'),pch=18,col=c('red','blue'))